# Using Solr tagger

https://lucene.apache.org/solr/guide/7_4/the-tagger-handler.html

In [ ]:
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from pandas.io.json import json_normalize
import requests



In [ ]:
options = {'url':'https://solr-dev.ideaconsult.net/solr/tagger/tag?tagsLimit=5000','name':'','password':''}
@interact(url=options['url'],name=options['name'], password=options['password'])
def g(url,name,password):
    options['url']=url
    options['name'] =name
    options['password'] = password
    return (url,name,password)

In [ ]:
# example query excerpt from https://arxiv.org/abs/1906.00212
query = "HRTEM, with EDX and high-angle annular dark-field scanning transmission electron microscopy were performed using \
JEOL JEMARM200F and JEM-ARM200CF with an acceleration voltage of 200 kV. The JEOL Analysis Station software was used to control the STEM-EDX mapping. \
To minimize the effect of sample drift, a driftcorrection mode was used during acquisition of the elemental map.\
The STEM probe size was ~0.13 nm, generating ~140 pA of current when 40 \u00b5m of the condenser lens aperture was inserted. \
The collection angle of the HAADF detector was ~97-256 mr"
@interact(query=query)
def g(query):
    x = requests.post(options['url'], data = query.encode(encoding='utf-8'),  auth=(options['name'],options['password']))
    if x.status_code==200:
        tags=x.json()["tags"]
        results = json_normalize(x.json()['response']['docs'])
        for tag in tags:
            #print(tag)
            _id=tag[5][0]
            term = results[results["id"].isin(tag[5])]      
            print("{}\t [at offset {}:{}]".format(query[tag[1]:tag[3]],tag[1],tag[3]))
            display(term[["id","name","ontology","definition"]])

        display(results)
        #print(x.json())
    else:
        display(x.status_code)  
    return (query)